# **Django REST Framework**

## **1. Django REST Framework 개요**

### 1.1 Django REST Framework(DRF)란?

- Django를 기반으로 RESTful API 서버를 만들기 위한 라이브러리
- RESTful API는 웹뿐만 아니라 앱과 같은 다양한 플랫폼의 백엔드 서비스를 위해 JSON과 같은 규격화된 데이터를 제공함
- `pip install djangorestframework` 명령으로 설치할 수 있음

### 1.2 주요 기능

#### 1.2.1 직렬화(Serialization)

- 모델 인스턴스를 JSON, XML 등 다양한 형식으로 변환할 수 있음
- `serializers.py` 파일을 통해 직렬화 로직을 정의함

  ```python
  from rest_framework import serializers
  from .models import Post

  class PostSerializer(serializers.ModelSerializer):
      class Meta:
          model = Post
          fields = '__all__'
  ```

#### 1.2.2 뷰(Views)

- DRF는 다양한 뷰 클래스를 제공하여 API 엔드포인트를 쉽게 정의할 수 있음
- `APIView`, `GenericAPIView`, `ViewSet` 등을 사용하여 다양한 요구사항을 충족할 수 있음

  ```python
  from rest_framework import generics
  from .models import Post
  from .serializers import PostSerializer

  class PostListCreate(generics.ListCreateAPIView):
      queryset = Post.objects.all()
      serializer_class = PostSerializer
  ```

#### 1.2.3 라우팅(Routing)

- Django의 URL 라우팅 시스템을 확장하여 API 엔드포인트를 쉽게 정의할 수 있음
    - 라우팅 시스템
        - 네트워크에서 데이터 패킷이 목적지까지 가장 효율적으로 전달될 수 있도록 경로를 선택하는 프로세스
        - 네트워크의 여러 노드(컴퓨터, 라우터 등) 간의 통신을 가능하게 하며, 인터넷과 같은 대규모 네트워크에서 특히 중요함
- `urls.py` 파일에서 라우터를 설정함

  ```python
  from django.urls import path, include
  from rest_framework.routers import DefaultRouter
  from .views import PostViewSet

  router = DefaultRouter()
  router.register(r'posts', PostViewSet)

  urlpatterns = [
      path('', include(router.urls)),
  ]
  ```

#### 1.2.4 인증 및 권한(Authentication and Permissions)

- 다양한 인증 방법(JWT, OAuth 등)을 지원하며, 사용자 권한을 세밀하게 설정할 수 있음
- `permissions.py` 파일에서 권한 로직을 정의함

  ```python
  from rest_framework import permissions

  class IsOwnerOrReadOnly(permissions.BasePermission):
      def has_object_permission(self, request, view, obj):
          if request.method in permissions.SAFE_METHODS:
              return True
          return obj.owner == request.user
  ```

#### 1.2.5 필터링 및 페이지네이션(Filter and Pagination)

- 쿼리셋을 필터링하고, 페이지네이션을 통해 데이터를 효율적으로 관리할 수 있음
- `views.py` 파일에서 필터 및 페이지네이션 설정을 추가함

  ```python
  from rest_framework import generics, filters
  from .models import Post
  from .serializers import PostSerializer

  class PostList(generics.ListAPIView):
      queryset = Post.objects.all()
      serializer_class = PostSerializer
      filter_backends = [filters.SearchFilter]
      search_fields = ['title', 'content']
  ```

## **2. Django REST Framework 예제 프로젝트**

### 2.1 프로젝트 생성

- 가상환경 설정

In [ ]:
python -m venv drf

In [ ]:
cd drf

In [ ]:
source ./bin/activate
# Windows의 경우: ./Scripts/activate

- Django 설치 및 확인

In [ ]:
pip install django djangorestframework

- Project 시작

In [ ]:
django-admin startproject drfweb .

- 앱 추가

In [ ]:
python manage.py startapp drf

In [ ]:
python manage.py migrate

- 웹서버 실행

In [ ]:
python manage.py runserver

- 웹 브라우저에서 웹서버 시작 확인
    - https://127.0.0.1:8000

- 웹서버 설정 수정

In [ ]:
# ./restweb/settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',   # 추가할 것
    'drf',          # 추가할 것
]

TIME_ZONE = 'Asia/Seoul'    # 시간대를 한국으로 바꿀 것

### 2.2 Django REST Framework 프로젝트 구조

- HelloAPI 만들기

In [ ]:
# ./drf/views.py

from rest_framework.response import Response
from rest_framework.decorators import api_view

# Create your views here.
@api_view(['GET'])
def helloAPI(request):
    return Response("hello world!")

- URL

In [ ]:
# ./drfweb/urls.py

from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path("drf/", include("drf.urls"))
]

In [ ]:
# ./drf/urls.py

from django.urls import path
from .views import helloAPI

urlpatterns = [
    path("hello/", helloAPI),
]

- REST API 확인
    - http://127.0.0.1:8000/drf/hello/

- Django vs DRF

|특징|Pure Django|Django REST Framework|
|------|---|---|
|개발 목적|웹 풀스택 개발|백엔드 API 서버 개발|
|개발 결과|웹 페이지를 포함한 웹 서비스|여러 클라이언트에서 사용할 수 있는 API 서버|
|응답 형태|HTML|JSON|
|다른 파일|templates|Serializers.py|

### 2.3 도서 정보 API

#### 2.3.1 Model

In [ ]:
# ./drf/models.py

from django.db import models

class Book(models.Model):
    bid = models.IntegerField(primary_key=True)
    title = models.CharField(max_length=50)
    author = models.CharField(max_length=50)
    category = models.CharField(max_length=50)
    pages = models.IntegerField()
    price = models.IntegerField()
    published_date = models.DateField()
    description = models.TextField()

- Model 적용시키기

In [ ]:
python manage.py makemigrations

In [ ]:
python manage.py migrate

#### 2.3.2 Serializer(직렬화)

- 직렬화(Serialization)란?
    - 객체를 데이터 스트림으로 변환하는 과정을 의미함
    - 이 과정을 통해 객체는 파일에 저장되거나 네트워크를 통해 전송될 수 있음
    - 직렬화는 주로 객체의 상태를 영속적으로 저장하거나 다른 시스템 간에 객체를 전송할 때 사용됨
    - Django의 경우 ORM 방식을 이용하여 파이썬이 사용하는 형태로 데이터를 저장하지만 클라이언트는 이를 이해하지 못하므로 클라이언트가 읽을 수 있는 JSON 등의 형태로 바꿔주는 기능이라고 볼 수 있음
<br><br>
- 직렬화의 주요 개념
    - 객체의 변환
        - 직렬화는 객체를 바이트 스트림으로 변환하여 파일에 저장하거나 네트워크를 통해 전송할 수 있게 함
        - 이를 통해 객체의 상태를 유지하면서 다른 환경에서도 동일한 객체를 사용할 수 있음
    - 역직렬화(Deserialization)
        - 직렬화된 데이터를 다시 객체로 변환하는 과정
        - 이를 통해 저장된 객체를 다시 메모리에 올릴 수 있음
        - 클라이언트가 보낸 데이터는 파이썬에서 바로 저장하거나 활용할 수 없으므로 다시 파이썬에서 사용하는 형태로 바꿔주는 기능이라고 할 수 있음
<br><br>
- 직렬화의 사용 예시
    - 객체의 영속화
        - 프로그램이 종료되더라도 객체의 상태를 유지하기 위해 파일에 저장할 수 있음
        - 예: 사용자 설정 정보를 직렬화하여 저장하면 프로그램 재실행 시에도 동일한 설정을 사용할 수 있음
    - 네트워크 통신
        - 직렬화된 객체를 네트워크를 통해 다른 시스템으로 전송할 수 있으며 이는 분산 시스템에서 중요한 역할을 함
        - 예: 클라이언트-서버 통신에서 객체를 직렬화하여 전송하고, 수신 측에서 역직렬화하여 객체를 복원할 수 있음


In [ ]:
# ./drf/serializers.py

from rest_framework import serializers
from .models import Book

class BookSerializer(serializers.ModelSerializer):
    class Meta:
        model = Book
        fields = ['bid', 'title', 'author', 'category', 'pages', 'price', 'published_date', 'description',]

- 일반적인 경우의 Serializer는
    - 파이썬 모델 데이터를 JSON으로 바꿔주는 변환기 이므로 모델 데이터의 어떤 속성을 JSON으로 넣어줄지 선언해야 함
    - 따라서 Serializer에도 필드를 선언해야 하므로 다음과 같이 복잡해지기 쉬움
    - 그래서 serializers.ModelSerializer를 사용하여 사용할 모델만을 기반으로 정의하는 방법을 사용하였음

In [ ]:
# 일반적인 경우 Serializer는 다음과 같이 복잡해짐

class BookSerializer(serializers.Serializer):
    bid = serializers.IntegerField()
    title = serializers.CharField(max_length=50)
    author = serializers.CharField(max_length=50)
    category = serializers.CharField(max_length=50)
    pages = serializers.IntegerField()
    price = serializers.IntegerField()
    published_date = serializers.DateField()
    description = serializers.TextField()

    def create(self, validated_data):
        return Book.objects.create(**validated_data)

    def update(self, instance, validated_data):
        instance.bid = validated_data.get('bid', instance.bid)
        instance.title = validated_data.get('title', instance.title)
        instance.author = validated_data.get('author', instance.author)
        instance.category = validated_data.get('category', instance.category)
        instance.pages = validated_data.get('pages', instance.pages)
        instance.price = validated_data.get('price', instance.price)
        instance.published_date = validated_data.get('published_date', instance.published_date)
        instance.description = validated_data.get('description', instance.description)
        instance.save()

        return instance

#### 2.3.3 DRF FBV, CBV, APIView

- FBV(Function Based View, 함수 기반 뷰), CBV(Class Based View, 클래스 기반 뷰)
    - 뷰를 작성할 때 함수를 사용했는지, 클래스를 사용했는지의 차이

- APIView
    - 여러 가지 유형의 요청에 대하여 FBV, CBV가 제대로 동작할 수 있도록 도와주는 역할
    - FBV에서 뷰를 만들 때
        - @api_view와 같이 데코레이터 형태로 생성
    - CBV에서 뷰를 만들 때
        - APIView를 상속받는 클래스의 형태로 생성

In [ ]:
# ./drf/views.py

from rest_framework.response import Response
from rest_framework.views import APIView
from rest_framework.decorators import api_view

@api_view(['GET'])
def helloAPI(request):
    return Response("hello world!")

class HelloAPI(APIView):
    def get(self, request, format=None):
        return Response("hello world")

- 데코레이터(Decorator)
    - 개념
        - 프로그래밍에서 데코레이터는 함수나 메소드의 동작을 수정하거나 확장하는 데 사용되는 특별한 함수를 의미함
        - 데코레이터는 다른 함수를 인자로 받아, 그 함수에 새로운 기능을 추가한 후, 수정된 함수를 반환하며, 이를 통해 코드의 재사용성을 높이고, 공통적인 패턴을 캡슐화하여 다양한 함수나 메소드에 적용할 수 있음
        - 데코레이터는 특히 로깅, 권한 검사, 성능 측정 등 다양한 상황에서 유용하게 사용될 수 있음
<br><br>
    - 데코레이터의 주요 역할
        - 코드 재사용성 향상: 공통 기능을 데코레이터로 정의하여 여러 함수에 쉽게 적용할 수 있음
        - 코드 가독성 향상: 함수의 핵심 로직과 부가 기능을 분리하여 코드가 더 깔끔하고 이해하기 쉬워짐
        - 유지보수 용이성: 공통 기능을 한 곳에서 관리할 수 있어, 수정이 필요할 때 여러 곳을 수정할 필요가 없음
<br><br>
    - 데코레이터의 예시
        ```python
        def my_decorator(func):
            def wrapper():
                print("함수 호출 전")
                func()
                print("함수 호출 후")
            return wrapper

        @my_decorator
        def say_hello():
            print("안녕하세요!")

        say_hello()
        ```
        - 위 코드에서 `my_decorator`는 `say_hello` 함수에 적용되어, `say_hello` 함수 호출 전후에 메시지를 출력하는 기능을 추가함


#### 2.3.4 Book API 만들기

In [ ]:
# ./drf/views.py

from rest_framework import status
from rest_framework.response import Response
from rest_framework.views import APIView
from rest_framework.decorators import api_view
from rest_framework.generics import get_object_or_404
from .models import Book
from .serializers import BookSerializer

@api_view(['GET', 'POST'])
def booksAPI(request):
    if request.method == 'GET':
        books = Book.objects.all()
        serializer = BookSerializer(books, many=True)
        return Response(serializer.data, status=status.HTTP_200_OK)
    elif request.method == 'POST':
        serializer = BookSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

@api_view(['GET'])
def bookAPI(request, bid):
    book = get_object_or_404(Book, bid=bid)
    serializer = BookSerializer(book)
    return Response(serializer.data, status=status.HTTP_200_OK)


class BooksAPI(APIView):
    def get(self, request):
        books = Book.objects.all()
        serializer = BookSerializer(books, many=True)
        return Response(serializer.data, status=status.HTTP_200_OK)
    def post(self, request):
        serializer = BookSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

class BookAPI(APIView):
    def get(self, request, bid):
        book = get_object_or_404(Book, bid=bid)
        serializer = BookSerializer(book)
        return Response(serializer.data, status=status.HTTP_200_OK)

#### 2.3.5 URL 연결

In [ ]:
# ./drf/urls.py

from django.urls import path
from .views import helloAPI, bookAPI, booksAPI, BookAPI, BooksAPI, BooksAPIMixins, BookAPIMixins

urlpatterns = [
    path("hello/", helloAPI),
    path("fbv/books/", booksAPI),
    path("fbv/book/<int:bid>/", bookAPI),
    path("cbv/books/", BooksAPI.as_view()),
    path("cbv/book/<int:bid>/", BookAPI.as_view()),
 ]

#### 2.3.6 데이터 확인하기

In [ ]:
{
    "bid": 1,
    "title": "처음 만나는 AI 수학 with 파이썬",
    "author": "아즈마 유키나가",
    "category": "프로그래밍",
    "pages": 308,
    "price": 20000,
    "published_date": "2021-01-30",
    "description": "인공지능을 공부하는데 필요한 기초 수학개념을 한 권에 모았다!"
}

In [ ]:
{
    "bid": 2,
    "title": "앱 인벤터, 상상을 현실로 만드는 프로젝트",
    "author": "이준혁",
    "category": "프로그래밍",
    "pages": 328,
    "price": 20000,
    "published_date": "2020-12-10",
    "description": "블록 코딩으로 만드는 안드로이드 앱, 앱 인벤터"
}